# TSA Chapter 5: VaR Backtesting Python Implementation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_backtest_py/TSA_ch5_backtest_py.ipynb)

Python implementation of VaR backtesting with statistical tests.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
import yfinance as yf
from arch import arch_model

In [ ]:
# Download Bitcoin data for backtesting
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc_returns = (btc['Close'].pct_change() * 100).dropna()
print(f"BTC returns: {len(btc_returns)} observations")

n_total = len(btc_returns); n_test = 250; n_train = n_total - n_test

In [ ]:
# Rolling GARCH VaR
var_1pct = []; var_5pct = []
for i in range(n_test):
    window = btc_returns.iloc[:n_train + i]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=1)
    sigma1 = np.sqrt(fcast.variance.values[-1, 0])
    var_1pct.append(2.326 * sigma1)
    var_5pct.append(1.645 * sigma1)
    if (i+1) % 50 == 0: print(f"  {i+1}/{n_test}")

test = btc_returns.iloc[n_train:]
v1 = (test.values < -np.array(var_1pct)).sum()
v5 = (test.values < -np.array(var_5pct)).sum()
print(f"\n1% VaR violations: {v1}/{n_test} ({v1/n_test:.3f})")
print(f"5% VaR violations: {v5}/{n_test} ({v5/n_test:.3f})")